$$
  \tag{1}
  C_p(T) = a_0 + a_1 T + a_2 \frac{1}{T^2}
$$

#### References
- Timothy K. Risch. "Curve Fits of the NIST-JANAF
Thermochemical Tables." NASA/TM—20205003319/VOL1. NASA. Feb 2021. https://ntrs.nasa.gov/api/citations/20205003319/downloads/H3289FinalVol%201.pdf



In [17]:
from dataclasses import dataclass
from enum import Enum

class JanafReferenceState(Enum):
    SOLID = "solid"
    LIQUID = "liquid"
    GAS = "gas"

@dataclass(frozen=True)
class JanafState:
    ref: JanafReferenceState
    charge: int  # 0, +1, -1

@dataclass(frozen=True)
class JanafEntry:
    dataset: str  # filepath (relative to data_root)


In [ ]:
# eq 1
def specific_heat_eqn(T, a0, a1, a2):
    """
    T: temperature (T)
    a0, a1, a2: fitting parameters

    returns:
    C_p: specific heat, cal/mol-K
    """
    C_p = a0 + a1 * T + a2 / T**2
    return C_p



# Janaf Files



In [ ]:
# https://janaf.nist.gov/tables/Si-002.txt
RAW = r"""Silicon (Si)	Si1(cr)
T(K)	Cp	S	-[G-H(Tr)]/T	H-H(Tr)	delta-f H	delta-f G	log Kf
0	0.	0.	INFINITE	-3.218	0.	0.	0.
100	7.268	3.833	33.351	-2.952	0.	0.	0.
200	15.636	11.665	20.531	-1.773	0.	0.	0.
250	18.221	15.446	19.138	-0.923	0.	0.	0.
298.15	20.000	18.820	18.820	0.	0.	0.	0.
300	20.050	18.943	18.820	0.037	0.	0.	0.
350	21.276	22.132	19.069	1.072	0.	0.	0.
400	22.142	25.032	19.636	2.159	0.	0.	0.
450	22.803	27.680	20.385	3.283	0.	0.	0.
500	23.330	30.110	21.237	4.436	0.	0.	0.
600	24.154	34.440	23.086	6.812	0.	0.	0.
700	24.803	38.212	24.983	9.260	0.	0.	0.
800	25.359	41.562	26.850	11.769	0.	0.	0.
900	25.874	44.579	28.655	14.331	0.	0.	0.
1000	26.338	47.329	30.387	16.942	0.	0.	0.
1100	26.778	49.860	32.044	19.598	0.	0.	0.
1200	27.196	52.208	33.627	22.297	0.	0.	0.
1300	27.614	54.401	35.142	25.037	0.	0.	0.
1400	28.033	56.463	36.592	27.820	0.	0.	0.
1500	28.451	58.411	37.982	30.644	0.	0.	0.
1600	28.870	60.261	39.317	33.510	0.	0.	0.
1685.000	29.225	61.765	40.412	35.979	CRYSTAL <--> LIQUID
1700	29.288	62.024	40.602	36.418	50.177 0.447  0.014
1800	29.706	63.710	41.839	39.368	-49.947	3.418	-0.099
1900	30.125	65.327	43.033	42.359	-49.675	6.376	-0.175
2000	30.543	66.883	44.187	45.393	-49.361	9.318	-0.243
2100	30.962	68.383	45.303	48.468	-49.006	12.243	-0.305
2200	31.380	69.833	46.386	51.585	-48.608	15.151	-0.360
2300	31.798	71.237	47.436	54.744	-48.169	18.039	-0.410
2400	32.217	72.600	48.456	57.945	-47.688	20.908	-0.455
2500	32.635	73.923	49.448	61.187	-47.165	23.755	-0.496"""



In [ ]:
import numpy as np
import pandas as pd

class JanafFile():
  janaf_col_names = [
    "T",
    "Cp",
    "S",
    "phi",
    "H_rel",
    "delta_f_H",
    "delta_f_G",
    "log_Kf"]
  
  def __init__(self):
    self.element_name: str = ""
    self.element_symbol: str = ""
    self.species_id: str = ""
    self.df: pd.DataFrame | None = None

  def process_line_0(self, tokens):
    self.element_name = tokens[0] if len(tokens) > 0 else ""
    self.element_symbol = tokens[1] if len(tokens) > 1 else ""
    self.species_id = tokens[2] if len(tokens) > 2 else ""

  def _try_float(self, tok: str):
    t = tok.strip()
    if t.upper() in {"INFINITE", "INFINITY", "INF"}:
      return np.inf
    t = t.replace("D", "E").replace("d", "E")  # Fortran exponent support
    try:
      return float(t)
    except ValueError:
      return None

  def process_line(self, tokens):
    ncols = len(self.janaf_col_names)
    nums: list[float] = []
    note_tokens: list[str] = []

    for tok in tokens:
      if len(nums) < ncols:
        v = self._try_float(tok)
        if v is None:
          note_tokens.append(tok)
        else:
          nums.append(v)
      else:
        note_tokens.append(tok)

    # pad to fixed width so df columns always match
    if len(nums) < ncols:
      nums.extend([np.nan] * (8 - len(nums)))
    elif len(nums) > ncols:
      # should be rare; keep provenance in notes and trim
      note_tokens.append(f"[extra_numeric={nums[8:]}]")
      nums = nums[:ncols]

    return nums, " ".join(note_tokens).strip()

  def parse_text(self,text: str):
    lines = [k.strip() for k in text.splitlines() if k.strip()]
    rows = []
    notes = []
    raw = []

    for i, line in enumerate(lines):
      tokens = line.split()

      if i == 0:
        self.process_line_0(tokens)
      elif i == 1:
        continue
      else:
        numbers, note = self.process_line(tokens)
        rows.append(numbers)
        notes.append(" ".join(note))
        raw.append(line)
    self.df = pd.DataFrame(
        data=rows, 
        columns=self.janaf_col_names
        )
    self.df["notes"] = notes
    self.df["raw"] = raw
    
  def parse_file(self, path):
    with open(path, "r") as f:
      self.parse_text(text=f.read())


In [ ]:
import os
from typing import Dict
def load_janaf_tables(
    registry: Dict[JanafState, JanafEntry], 
    data_root: str = "."
) -> Dict[JanafState, JanafFile]:
  """
  Load JANAF tables defined by a registry mapping state -> filename.

  Parameters
  ----------
  registry:
      Dict mapping JanafState(ref, charge) to a filename (relative to data_root).
  data_root:
      Base folder containing the files. If None, uses this module's folder.
  verbose:
      If True, prints what was loaded.

  Returns
  -------
  Dict[JanafState, JanafFile]
      Parsed tables keyed by state.
  """
  tables: Dict[JanafState, JanafFile] = {}

  for state, entry in registry.items():
    path = os.path.join(data_root, entry.dataset)
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Missing JANAF file: {path}")

    jf = JanafFile()
    jf.parse_file(path)  # NOTE: pass the resolved path
    tables[state] = jf

    print(f"Loaded {state.ref.value}, charge={state.charge}: {path}")

  return tables

JANAF_AL: dict[JanafState, JanafEntry] = {
    JanafState(JanafReferenceState.SOLID, 0): JanafEntry("janafnist_Al_cr.txt"),
    JanafState(JanafReferenceState.LIQUID, 0): JanafEntry("janafnist_Al_l.txt"),
    JanafState(JanafReferenceState.GAS, 0): JanafEntry("janafnist_Al_g.txt"),
    JanafState(JanafReferenceState.GAS, +1): JanafEntry("janafnist_Al_g_ion_p.txt"),
    JanafState(JanafReferenceState.GAS, -1): JanafEntry("janafnist_Al_g_ion_n.txt"),
}

tables_Al = load_janaf_tables(JANAF_AL, data_root=".")

Loaded solid, charge=0: ./janafnist_Al_cr.txt
Loaded liquid, charge=0: ./janafnist_Al_l.txt
Loaded gas, charge=0: ./janafnist_Al_g.txt
Loaded gas, charge=1: ./janafnist_Al_g_ion_p.txt
Loaded gas, charge=-1: ./janafnist_Al_g_ion_n.txt
